<a href="https://colab.research.google.com/github/EmaMule/Computer-Vision/blob/main/GroundDepthMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!git clone https://github.com/LiheYoung/Depth-Anything
!pip install -q -r /content/Depth-Anything/requirements.txt
!pip install -q spaces

In [ ]:
cd Depth-Anything

/content/Depth-Anything


In [ ]:
from google.colab import drive


from tqdm import tqdm
import gc

import gradio as gr
import cv2
import numpy as np
import os
from PIL import Image
import spaces
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose
import tempfile
from gradio_imageslider import ImageSlider

from depth_anything.dpt import DepthAnything
from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Model Creation

In [ ]:
@spaces.GPU
@torch.no_grad()
def predict_depth(model, image):
    return model(image)


def obtain_depth_map(model, transform, img_path):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    image = Image.open(img_path)
    image = np.array(image)

    original_image = image.copy()

    h, w = image.shape[:2]

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0
    image = transform({'image': image})['image']
    image = torch.from_numpy(image).unsqueeze(0).to(device)

    depth = predict_depth(model, image)
    depth = F.interpolate(depth[None], (h, w), mode='bilinear', align_corners=False)[0, 0]

    raw_depth = Image.fromarray(depth.cpu().numpy().astype('uint16'))
    tmp = tempfile.NamedTemporaryFile(suffix='.png', delete=False)
    raw_depth.save(tmp.name)

    depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
    depth = depth.cpu().numpy().astype(np.uint8)
    colored_depth = cv2.applyColorMap(depth, cv2.COLORMAP_INFERNO)[:, :, ::-1]

    return colored_depth

In [ ]:
dir_path = "/content/drive/MyDrive/CV Project/CVUSA/streetview"
image_names = os.listdir(dir_path)

image_paths = []
for image_name in image_names:
    image_path = os.path.join(dir_path, image_name)
    image_paths.append(image_path)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
encoder = 'vitl'
model = DepthAnything.from_pretrained(f"LiheYoung/depth_anything_{encoder}14").to(device).eval()

transform = Compose([
        Resize(
            width=518,
            height=518,
            resize_target=False,
            keep_aspect_ratio=True,
            ensure_multiple_of=14,
            resize_method='lower_bound',
            image_interpolation_method=cv2.INTER_CUBIC,
        ),
        NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        PrepareForNet(),
])

# Example

In [ ]:
Image.open(image_paths[10])

In [ ]:
colored_depth = obtain_depth_map(model, transform, image_paths[10])
depth_image = Image.fromarray(colored_depth)
depth_image

# Dataset Creation

In [ ]:
target_dir_path = "/content/drive/MyDrive/CV Project/CVUSA/streetview_depth"

if not os.path.exists(target_dir_path):
    os.makedirs(target_dir_path)

depth_paths = []
for image_name in image_names:
    image_path = os.path.join(target_dir_path, image_name)
    depth_paths.append(image_path)

In [ ]:
colored_depths = []
for i in tqdm(range(len(image_paths))):
    if i % 200 == 0:
        gc.collect()
        torch.cuda.empty_cache()
    if not os.path.exists(depth_paths[i]):
        colored_depth = obtain_depth_map(model, transform, image_paths[i])
        depth_image = Image.fromarray(colored_depth)
        depth_image.save(depth_paths[i])